<a href="https://colab.research.google.com/github/Linda-lls/GEOG0105/blob/main/8_Validation_of_IVW_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Validation of IVW**

##Selection of sample points for validation

In [ ]:
import csv
import random
import shutil
import os
import pandas as pd
import numpy as np
from PIL import Image
from scipy import stats

In [ ]:
# Import IVW
IVW = pd.read_csv("/content/drive/MyDrive/GEOG0105/Output_1/SegFormer/IVW.csv")

In [ ]:
# select 40*5% = 2 sample points from each ward for validation
random.seed(1)
validation_points = IVW.groupby('WARD_ID').apply(lambda x: x.sample(2)).reset_index(drop=True)

In [ ]:
print(validation_points.info())
validation_points

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_x         260 non-null    int64  
 1   lon          260 non-null    float64
 2   lat          260 non-null    float64
 3   geometry     260 non-null    object 
 4   index_right  260 non-null    int64  
 5   OBJECTID     260 non-null    int64  
 6   PROVINCE     260 non-null    object 
 7   CAT_B        260 non-null    object 
 8   WARDNO       260 non-null    int64  
 9   WARD_ID      260 non-null    int64  
 10  SUM_WARD_P   260 non-null    int64  
 11  SHAPE_LENG   260 non-null    float64
 12  WARD_EXT     260 non-null    object 
 13  ALBERS_ARE   260 non-null    float64
 14  SHAPE_Le_1   260 non-null    float64
 15  SHAPE_Area   260 non-null    float64
 16  T            260 non-null    float64
 17  C            260 non-null    float64
 18  B            260 non-null    float64
 19  P       

,id_x,lon,lat,geometry,index_right,OBJECTID,PROVINCE,CAT_B,WARDNO,WARD_ID,...,F,Gi,Ci,Si,Di,Gi_level,Ci_level,Si_level,Di_level,IVW
0,20,27.856629,-26.484906,POINT (27.8566293 -26.48490555),1392,1306,Gauteng,JHB,1,79800001,...,0.000117,0.018151,0.000586,1.419580,0.001911,1,5,5,1,60
1,21,27.842154,-26.486767,POINT (27.842154 -26.48676665),1392,1306,Gauteng,JHB,1,79800001,...,0.002773,0.051224,0.005456,0.228769,0.002827,1,5,1,1,40
2,79,27.879742,-26.517191,POINT (27.8797425 -26.5171913),1768,1307,Gauteng,JHB,2,79800002,...,0.065508,0.005221,0.003529,0.123366,0.068014,1,5,1,1,40
3,40,27.875915,-26.512709,POINT (27.87591508859661 -26.512709332175856),1768,1307,Gauteng,JHB,2,79800002,...,0.008047,0.146276,0.002969,0.515052,0.021434,3,5,1,1,50
4,89,27.863384,-26.485634,POINT (27.8633837 -26.4856343),1393,1308,Gauteng,JHB,3,79800003,...,0.001289,0.001836,0.007214,0.211357,0.067453,1,5,1,1,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,5085,27.798402,-26.184734,POINT (27.79840195 -26.18473365),1892,1433,Gauteng,JHB,128,79800128,...,0.007539,0.033424,0.001680,0.198465,0.041251,1,5,1,1,40
256,5138,27.827045,-26.225972,POINT (27.827045317735948 -26.22597208293208),2008,1434,Gauteng,JHB,129,79800129,...,0.000000,0.015911,0.000234,0.264057,0.113578,1,5,1,1,40
257,5136,27.826277,-26.232765,POINT (27.826277154970345 -26.232764919855068),2008,1434,Gauteng,JHB,129,79800129,...,0.009297,0.001953,0.004128,0.319883,0.139891,1,5,1,1,40
258,5186,27.822272,-26.256015,POINT (27.82227191475408 -26.2560152762325),2009,1435,Gauteng,JHB,130,79800130,...,0.044805,0.072096,0.024492,0.409765,0.049186,2,5,1,1,45


In [ ]:
# save validation points as csv
validation_points.to_csv('/content/drive/MyDrive/GEOG0105/Output_1/Validation/validation_points.csv', index=True)

In [ ]:
# save validation figures in folder GSV_validation

source_folder = "/content/drive/MyDrive/GEOG0105/GSV_final"

target_folder = "/content/drive/MyDrive/GEOG0105/Output_1/Validation/GSV_validation"

for index, row in validation_points.iterrows():
    id = row['id_x']
    ward = row['WARD_ID']

    # 根据样本点编号和角度复制照片
    for angle in range(3):  # 假设有3个角度的照片
        source_filename = os.path.join(source_folder, f"{id}_angle_{angle}.jpg.jpg")
        target_filename = os.path.join(target_folder, f"{ward}_{id}_angle_{angle}.jpg")

        shutil.copy(source_filename, target_filename)

print("Copy complete!")

Copy complete!


## Combine images from same sample points as 360 view GSV

In [ ]:
input_folder = "/content/drive/MyDrive/GEOG0105/Output_1/Validation/GSV_validation"
output_folder = "/content/drive/MyDrive/GEOG0105/Output_1/Validation/GSV_val_360"
os.makedirs(output_folder, exist_ok=True)

gap_width = 10

angle_0_img = None
angle_1_img = None
angle_2_img = None

for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):
        parts = filename.split("_")
        ward_id, idx, angle = int(parts[0]), int(parts[1]), int(parts[3][0])
        target_filename = os.path.join(output_folder, f"{ward_id}_{idx}.jpg")

        img = Image.open(os.path.join(input_folder, filename))

        if angle == 0:
            angle_0_img = img
        elif angle == 1:
            angle_1_img = img
        elif angle == 2:
            angle_2_img = img
            if angle_0_img and angle_1_img:
                combined_img = Image.new("RGB", (angle_0_img.width + angle_1_img.width + angle_2_img.width + 2 * gap_width, angle_0_img.height), (255, 255, 255))
                x_offset = 0
                combined_img.paste(angle_2_img, (x_offset, 0))
                x_offset += angle_2_img.width + gap_width
                combined_img.paste(angle_0_img, (x_offset, 0))
                x_offset += angle_0_img.width + gap_width
                combined_img.paste(angle_1_img, (x_offset, 0))
                combined_img.save(target_filename)
                print(f"saved {target_filename}")

print("combination has been finished")

## Pearson Correlation between IVW and subjective scoring

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import validation points
validation_points = pd.read_csv("/content/drive/MyDrive/GEOG0105/Output_1/Validation/validation_points.csv")
print(validation_points.info())
validation_points.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   260 non-null    int64  
 1   id_x         260 non-null    int64  
 2   lon          260 non-null    float64
 3   lat          260 non-null    float64
 4   geometry     260 non-null    object 
 5   index_right  260 non-null    int64  
 6   OBJECTID     260 non-null    int64  
 7   PROVINCE     260 non-null    object 
 8   CAT_B        260 non-null    object 
 9   WARDNO       260 non-null    int64  
 10  WARD_ID      260 non-null    int64  
 11  SUM_WARD_P   260 non-null    int64  
 12  SHAPE_LENG   260 non-null    float64
 13  WARD_EXT     260 non-null    object 
 14  ALBERS_ARE   260 non-null    float64
 15  SHAPE_Le_1   260 non-null    float64
 16  SHAPE_Area   260 non-null    float64
 17  T            260 non-null    float64
 18  C            260 non-null    float64
 19  B       

,Unnamed: 0,id_x,lon,lat,geometry,index_right,OBJECTID,PROVINCE,CAT_B,WARDNO,...,F,Gi,Ci,Si,Di,Gi_level,Ci_level,Si_level,Di_level,IVW
0,0,20,27.856629,-26.484906,POINT (27.8566293 -26.48490555),1392,1306,Gauteng,JHB,1,...,0.000117,0.018151,0.000586,1.419580,0.001911,1,5,5,1,60
1,1,21,27.842154,-26.486767,POINT (27.842154 -26.48676665),1392,1306,Gauteng,JHB,1,...,0.002773,0.051224,0.005456,0.228769,0.002827,1,5,1,1,40
2,2,79,27.879742,-26.517191,POINT (27.8797425 -26.5171913),1768,1307,Gauteng,JHB,2,...,0.065508,0.005221,0.003529,0.123366,0.068014,1,5,1,1,40
3,3,40,27.875915,-26.512709,POINT (27.87591508859661 -26.512709332175856),1768,1307,Gauteng,JHB,2,...,0.008047,0.146276,0.002969,0.515052,0.021434,3,5,1,1,50
4,4,89,27.863384,-26.485634,POINT (27.8633837 -26.4856343),1393,1308,Gauteng,JHB,3,...,0.001289,0.001836,0.007214,0.211357,0.067453,1,5,1,1,40


In [ ]:
IVW_ranges = [0, 35, 55, 70, 80, 100]
IVW_levels = [1, 2, 3, 4, 5]
validation_points['IVW_level'] = pd.cut(validation_points['IVW'], bins=IVW_ranges, labels=IVW_levels, right=True)
validation_points.head()

,Unnamed: 0,id_x,lon,lat,geometry,index_right,OBJECTID,PROVINCE,CAT_B,WARDNO,...,Gi,Ci,Si,Di,Gi_level,Ci_level,Si_level,Di_level,IVW,IVW_level
0,0,20,27.856629,-26.484906,POINT (27.8566293 -26.48490555),1392,1306,Gauteng,JHB,1,...,0.018151,0.000586,1.419580,0.001911,1,5,5,1,60,3
1,1,21,27.842154,-26.486767,POINT (27.842154 -26.48676665),1392,1306,Gauteng,JHB,1,...,0.051224,0.005456,0.228769,0.002827,1,5,1,1,40,2
2,2,79,27.879742,-26.517191,POINT (27.8797425 -26.5171913),1768,1307,Gauteng,JHB,2,...,0.005221,0.003529,0.123366,0.068014,1,5,1,1,40,2
3,3,40,27.875915,-26.512709,POINT (27.87591508859661 -26.512709332175856),1768,1307,Gauteng,JHB,2,...,0.146276,0.002969,0.515052,0.021434,3,5,1,1,50,2
4,4,89,27.863384,-26.485634,POINT (27.8633837 -26.4856343),1393,1308,Gauteng,JHB,3,...,0.001836,0.007214,0.211357,0.067453,1,5,1,1,40,2


In [ ]:
# Import subjective scoring results
subjective_scoring = pd.read_csv("/content/drive/MyDrive/GEOG0105/Output_1/Validation/Visual Walkability Scoring from Pedestrians' View_reply_record.csv")
print(subjective_scoring.info())
subjective_scoring.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   index      260 non-null    int64
 1   id         260 non-null    int64
 2   Scoring_1  260 non-null    int64
 3   Scoring_2  260 non-null    int64
 4   Scoring_3  260 non-null    int64
 5   Scoring_4  260 non-null    int64
dtypes: int64(6)
memory usage: 12.3 KB
None


,index,id,Scoring_1,Scoring_2,Scoring_3,Scoring_4
0,0,20,3,3,2,2
1,1,21,3,2,2,2
2,2,40,2,2,2,2
3,3,79,2,2,2,3
4,4,89,2,3,2,2


In [ ]:
# Calculate mean of 4 scoring results
subjective_scoring['mean'] = subjective_scoring[['Scoring_1', 'Scoring_2', 'Scoring_3', 'Scoring_4']].mean(axis=1)

subjective_scoring.head()

,index,id,Scoring_1,Scoring_2,Scoring_3,Scoring_4,mean
0,0,20,3,3,2,2,2.50
1,1,21,3,2,2,2,2.25
2,2,40,2,2,2,2,2.00
3,3,79,2,2,2,3,2.25
4,4,89,2,3,2,2,2.25


In [ ]:
val_sub = pd.merge(validation_points, subjective_scoring, left_on='id_x', right_on='id')
print(val_sub.info())
val_sub

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260 entries, 0 to 259
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Unnamed: 0   260 non-null    int64   
 1   id_x         260 non-null    int64   
 2   lon          260 non-null    float64 
 3   lat          260 non-null    float64 
 4   geometry     260 non-null    object  
 5   index_right  260 non-null    int64   
 6   OBJECTID     260 non-null    int64   
 7   PROVINCE     260 non-null    object  
 8   CAT_B        260 non-null    object  
 9   WARDNO       260 non-null    int64   
 10  WARD_ID      260 non-null    int64   
 11  SUM_WARD_P   260 non-null    int64   
 12  SHAPE_LENG   260 non-null    float64 
 13  WARD_EXT     260 non-null    object  
 14  ALBERS_ARE   260 non-null    float64 
 15  SHAPE_Le_1   260 non-null    float64 
 16  SHAPE_Area   260 non-null    float64 
 17  T            260 non-null    float64 
 18  C            260 non-null    f

,Unnamed: 0,id_x,lon,lat,geometry,index_right,OBJECTID,PROVINCE,CAT_B,WARDNO,...,Di_level,IVW,IVW_level,index,id,Scoring_1,Scoring_2,Scoring_3,Scoring_4,mean
0,0,20,27.856629,-26.484906,POINT (27.8566293 -26.48490555),1392,1306,Gauteng,JHB,1,...,1,60,3,0,20,3,3,2,2,2.50
1,1,21,27.842154,-26.486767,POINT (27.842154 -26.48676665),1392,1306,Gauteng,JHB,1,...,1,40,2,1,21,3,2,2,2,2.25
2,2,79,27.879742,-26.517191,POINT (27.8797425 -26.5171913),1768,1307,Gauteng,JHB,2,...,1,40,2,3,79,2,2,2,3,2.25
3,3,40,27.875915,-26.512709,POINT (27.87591508859661 -26.512709332175856),1768,1307,Gauteng,JHB,2,...,1,50,2,2,40,2,2,2,2,2.00
4,4,89,27.863384,-26.485634,POINT (27.8633837 -26.4856343),1393,1308,Gauteng,JHB,3,...,1,40,2,4,89,2,3,2,2,2.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,255,5085,27.798402,-26.184734,POINT (27.79840195 -26.18473365),1892,1433,Gauteng,JHB,128,...,1,40,2,254,5085,2,2,2,2,2.00
256,256,5138,27.827045,-26.225972,POINT (27.827045317735948 -26.22597208293208),2008,1434,Gauteng,JHB,129,...,1,40,2,257,5138,2,2,2,3,2.25
257,257,5136,27.826277,-26.232765,POINT (27.826277154970345 -26.232764919855068),2008,1434,Gauteng,JHB,129,...,1,40,2,256,5136,2,2,3,2,2.25
258,258,5186,27.822272,-26.256015,POINT (27.82227191475408 -26.2560152762325),2009,1435,Gauteng,JHB,130,...,1,45,2,258,5186,2,3,3,2,2.50


In [ ]:
# save merged validation_points and subjective_scoring as csv
val_sub.to_csv('/content/drive/MyDrive/GEOG0105/Output_1/Validation/val_sub_update.csv', index=False)

In [ ]:
# Pearson Correlation
res = stats.pearsonr(val_sub['IVW_level'], val_sub['mean'])
res

PearsonRResult(statistic=0.47553715363729454, pvalue=4.474590745388828e-16)